In [20]:
import os
import sys
import time
import pickle
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from tqdm import tqdm_notebook as tqdm
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, Activation, Input, GlobalAveragePooling2D
from keras.models import Model
from keras.utils import to_categorical
from keras.optimizers import SGD, RMSprop
from keras.applications.inception_v3 import InceptionV3, preprocess_input


# from google.colab import drive
# drive.mount("/content/gdrive")

In [21]:
drive_base_path = "../"
path_to_images = drive_base_path + "dataset/ISIC2018_Balanced_Training_Input/"

In [27]:
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    data_format="channels_last")

In [ ]:
classes = ['mel', 'nv', 'bcc', 'akiec', 'bkl', 'df', 'vasc']
train_generator = datagen.flow_from_directory(
                    directory=path_to_images,
                    target_size=(299, 299),
                    batch_size=32,
                    classes=classes,
                    shuffle=True)

In [ ]:
class_weights = class_weight.compute_class_weight(
               'balanced',
                np.unique(train_generator.classes), 
                train_generator.classes)

print(class_weights)

# DEFINING THE MODEL

In [28]:
base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(299, 299, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2048, activation='relu')(x)
predictions = Dense(7, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(loss="categorical_crossentropy", optimizer=RMSprop(lr=1e-5), metrics=["accuracy"])
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_283 (Conv2D)             (None, 149, 149, 32) 864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_283 (BatchN (None, 149, 149, 32) 96          conv2d_283[0][0]                 
__________________________________________________________________________________________________
activation_283 (Activation)     (None, 149, 149, 32) 0           batch_normalization_283[0][0]    
__________________________________________________________________________________________________
conv2d_284

In [29]:
batch_size = 32
epochs = 2
history = model.fit_generator(train_generator, 
                              epochs=epochs,
                              class_weight=class_weights,
                              steps_per_epoch=520,
                              verbose=1)

Epoch 1/2
313/312 [==============================] - 70s 222ms/step - loss: 1.0168 - acc: 0.6703
Epoch 2/2
313/312 [==============================] - 61s 196ms/step - loss: 0.8884 - acc: 0.6898


In [30]:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

0 input_4
1 conv2d_283
2 batch_normalization_283
3 activation_283
4 conv2d_284
5 batch_normalization_284
6 activation_284
7 conv2d_285
8 batch_normalization_285
9 activation_285
10 max_pooling2d_13
11 conv2d_286
12 batch_normalization_286
13 activation_286
14 conv2d_287
15 batch_normalization_287
16 activation_287
17 max_pooling2d_14
18 conv2d_291
19 batch_normalization_291
20 activation_291
21 conv2d_289
22 conv2d_292
23 batch_normalization_289
24 batch_normalization_292
25 activation_289
26 activation_292
27 average_pooling2d_28
28 conv2d_288
29 conv2d_290
30 conv2d_293
31 conv2d_294
32 batch_normalization_288
33 batch_normalization_290
34 batch_normalization_293
35 batch_normalization_294
36 activation_288
37 activation_290
38 activation_293
39 activation_294
40 mixed0
41 conv2d_298
42 batch_normalization_298
43 activation_298
44 conv2d_296
45 conv2d_299
46 batch_normalization_296
47 batch_normalization_299
48 activation_296
49 activation_299
50 average_pooling2d_29
51 conv2d_295
52

In [31]:
model.compile(optimizer=SGD(lr=1e-5, momentum=0.9), loss='categorical_crossentropy', metrics=["accuracy"])
epochs = 10
batch_size = 32
history = model.fit_generator(train_generator, 
                              steps_per_epoch=520,
                              epochs=epochs,
                              class_weight=class_weights,
                              verbose=1)
model.save_weights(drive_base_path + 'models/InceptionV3Net_ISIC_cnn.h5')

Epoch 1/10
313/312 [==============================] - 86s 275ms/step - loss: 0.8481 - acc: 0.7053
Epoch 2/10
313/312 [==============================] - 76s 242ms/step - loss: 0.8455 - acc: 0.7050
Epoch 3/10
313/312 [==============================] - 76s 242ms/step - loss: 0.8710 - acc: 0.6960
Epoch 4/10
313/312 [==============================] - 76s 242ms/step - loss: 1.0418 - acc: 0.6705
Epoch 5/10
313/312 [==============================] - 76s 242ms/step - loss: 1.1316 - acc: 0.6695
Epoch 6/10
313/312 [==============================] - 76s 242ms/step - loss: 1.1315 - acc: 0.6695
Epoch 7/10
313/312 [==============================] - 76s 242ms/step - loss: 1.1315 - acc: 0.6695
Epoch 8/10
313/312 [==============================] - 76s 242ms/step - loss: 1.1316 - acc: 0.6695
Epoch 9/10
313/312 [==============================] - 76s 242ms/step - loss: 1.1314 - acc: 0.6695
Epoch 10/10
116/312 [==========>...................] - ETA: 47s - loss: 1.1274 - acc: 0.6691

KeyboardInterrupt: 

# ACCURACY AND LOSS PLOTS

In [0]:
# Plot Training and Validation Accuracy
plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# CONFUSION MATRIX

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

y_test = y_train[:500]
X_test = X_train[:500]

y_pred = model.predict_generator(datagen.flow(X_test, batch_size=1), steps=len(X_test))

In [ ]:
y_pred = (y_pred > 0.5).astype(np.int)
y_test = y_test.argmax(axis=1)
y_pred = y_pred.argmax(axis=1)

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    #classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
class_names = ["MEL", "NV", "BCC", "AKIEC", "BKL", "DF", "VASC"]
plot_confusion_matrix(y_test, y_pred, classes=class_names, title='Confusion matrix, without normalization')
plt.show()

# ISIC LIVE IMAGE TEST/VALIDATION DATA

In [0]:
path_to_images = drive_base_path + "dataset/ISIC2018_Task3_Validation_Input/"

In [0]:
list_of_images = os.listdir(path_to_images)
filename = list_of_images[0]
print(filename[:-4])
input_size = 224

image = cv2.imread(path_to_images + filename)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
old_size = image.shape[:2]

ratio = float(input_size)/max(old_size)
new_size = tuple([int(x * ratio) for x in old_size])

image = cv2.resize(image, (new_size[1], new_size[0]))

delta_w = input_size - new_size[1]
delta_h = input_size - new_size[0]
top, bottom = delta_h//2, delta_h-(delta_h//2)
left, right = delta_w//2, delta_w-(delta_w//2)

color = [0, 0, 0]
new_img = cv2.copyMakeBorder(image, top, bottom, left, right, 
                             cv2.BORDER_CONSTANT, value=color)

plt.imshow(new_img)


In [0]:
def preprocess_images(path_to_images):
    images_list = []
    filename_list = []
    
    input_size = 224
    list_of_images = os.listdir(path_to_images)
    number = len(list_of_images)
    
    for filename in tqdm(list_of_images, total=number):
        filename_list.append(filename[:-4])
        
        image = cv2.imread(path_to_images + filename)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        old_size = image.shape[:2]
        
        ratio = float(input_size)/max(old_size)
        new_size = tuple([int(x * ratio) for x in old_size])
        
        image = cv2.resize(image, (new_size[1], new_size[0]))
        
        delta_w = input_size - new_size[1]
        delta_h = input_size - new_size[0]
        top, bottom = delta_h//2, delta_h-(delta_h//2)
        left, right = delta_w//2, delta_w-(delta_w//2)
        
        color = [0, 0, 0]
        new_img = cv2.copyMakeBorder(image, top, bottom, left, right, 
                                     cv2.BORDER_CONSTANT, value=color)

#         arr = np.array(image)
        images_list.append(new_img)
        
    images_list = np.array(images_list)
    filename_list = np.array(filename_list)
    filename_list = filename_list[:, None]
    return (images_list, filename_list)

In [0]:
x, names = preprocess_images(path_to_images)

In [0]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=0,
    width_shift_range=0.0,
    height_shift_range=0.0,
    horizontal_flip=False,
    rescale=1. /255.,
    fill_mode='nearest',
    data_format="channels_last",
    shuffle=False)

datagen.fit(x)

In [0]:
print("X Shape", x.shape)
plt.imshow(x[60])
print("Name Shape", names.shape)

In [0]:
model.load_weights(drive_base_path + 'models/InceptionV3Net_ISIC_cnn.h5')

In [0]:
predictions = model.predict_generator(datagen.flow(x, batch_size=1))
classes = (predictions > 0.5).astype(np.float)
print(classes.shape)

In [0]:
final_output = np.concatenate((names, classes), axis=1)
final_output.shape

In [0]:
import csv

with open("output.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["image", "MEL", "NV", "BCC", "AKIEC", "BKL", "DF", "VASC"])
    writer.writerows(final_output)